<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span><ul class="toc-item"><li><span><a href="#Выводы" data-toc-modified-id="Выводы-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li></ul></div>

# Выбор локации для скважины

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Цель**  

Построение модели для определения региона, где добыча нефти принесёт наибольшую прибыль. 

**Структура исследования**  

1. Загрузка и подготовка данных
2. Обучение и проверка модели
3. Подготовка к расчёту прибыли
4. Расчёт прибыли и рисков

**Использование дополнительных модулей**  

- `pandas` – редактор баз данных  
- `numpy` – работа с многомерными массивами
- `sklearn` – машинное обучение

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
# Загрузка датафреймов
df_0 = pd.read_csv('geo_data_0.csv')
df_1 = pd.read_csv('geo_data_1.csv')
df_2 = pd.read_csv('geo_data_2.csv')

In [3]:
# Получение общих сведений о датафрейме
df_0.info()
print('='*40)
df_1.info()
print('='*40)
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

In [4]:
# Проверка отсутствия дубликатов
print(f'Пропуски в датафрейме df_0: {"отсутствуют" if df_0.duplicated().sum() == 0 else "обнаружены"}')
print(f'Пропуски в датафрейме df_1: {"отсутствуют" if df_1.duplicated().sum() == 0 else "обнаружены"}')
print(f'Пропуски в датафрейме df_2: {"отсутствуют" if df_2.duplicated().sum() == 0 else "обнаружены"}')

Пропуски в датафрейме df_0: отсутствуют
Пропуски в датафрейме df_1: отсутствуют
Пропуски в датафрейме df_2: отсутствуют


In [5]:
# Пример сводной статистики для первого 'df_0' датафрейма
df_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


### Вывод

Выполнена загрузка и первичный анализ имеющихся датафреймов. Данные представлены в однородном виде, пропуски отсутствуют.

## Обучение и проверка модели

Обучение модели линейной регрессии для региона `1` (датафрейм `df_0`), `2` (датафрейм `df_1`) и `3` (датафрейм `df_2`).

In [6]:
def split_and_model(df):
    
    features = df.drop(['id', 'product'], axis=1)
    target = df['product']

    features_train, features_val, target_train, target_val = train_test_split(features, 
                                                                              target, test_size=0.25, random_state=12345)

    # Обучение модели и предсказание
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_val = model.predict(features_val)
    
    return predicted_val, target_val

In [7]:
predicted_val_1, target_val_1 = split_and_model(df_0)
predicted_val_2, target_val_2 = split_and_model(df_1)
predicted_val_3, target_val_3 = split_and_model(df_2)

for i in range(1, 4):
    print(f'Средний запас предсказанного сырья (регион {eval(f"{i}")}): {eval(f"predicted_val_{i}").mean():.0f} тыс. баррелей')
    print(f'RMSE (регион {eval(f"{i}")}): {mean_squared_error(eval(f"target_val_{i}"), eval(f"predicted_val_{i}"))**0.5:.1f}')
    print('='*30)

Средний запас предсказанного сырья (регион 1): 93 тыс. баррелей
RMSE (регион 1): 37.6
Средний запас предсказанного сырья (регион 2): 69 тыс. баррелей
RMSE (регион 2): 0.9
Средний запас предсказанного сырья (регион 3): 95 тыс. баррелей
RMSE (регион 3): 40.0


### Выводы

Регион `2` имеет наименьшую погрешность метрики `RMSE`, при этом также минимальный средний запас предсказанного сырья. В регионах `1` и `3`, показатели контрольной метрики и среднего запаса предсказанного сырья приблизительно сопоставимы. При этом погрешность метрики RMSE приблизительно 40 раз выше, средний запас предсказанного сырья примерно в 1,4 раза больше.

## Подготовка к расчёту прибыли

In [8]:
# Из условий задания:
    
# бюджет на разработку скважин в регионе, руб
TOTAL_BUDGET = 10_000_000_000

# Количество исследуемых точек
OIL_WELS = 500

# число выбранных для разработки точек
SELECTED_OIL_WELS = 200

# доход с каждой единицы продукта, руб
INCOME = 450_000

In [9]:
# Затраты на бурение одной скважины
drilling_cost = TOTAL_BUDGET / SELECTED_OIL_WELS

# Сырьевая безубыточность разработки одной новой скважины
breakeven_point = drilling_cost / INCOME

print(f'Достаточный объём сырья для безубыточной разработки одной новой скважины: {breakeven_point:.0f} тыс. баррелей')

Достаточный объём сырья для безубыточной разработки одной новой скважины: 111 тыс. баррелей


In [10]:
# Средний запас сырья в каждом регионе
print(f'Средний запас сырья в регионе 1: {df_0["product"].mean():.0f} тыс. баррелей')
print(f'Средний запас сырья в регионе 2: {df_1["product"].mean():.0f} тыс. баррелей')
print(f'Средний запас сырья в регионе 3: {df_2["product"].mean():.0f} тыс. баррелей')

Средний запас сырья в регионе 1: 93 тыс. баррелей
Средний запас сырья в регионе 2: 69 тыс. баррелей
Средний запас сырья в регионе 3: 95 тыс. баррелей


### Вывод

В каждом из исследованных регионов средний запас сырья не превышает сырьевую безубыточность разработки новой скважины.

## Расчёт прибыли и рисков 

**п. 4.1**  

Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [11]:
def profit(predicted_sub, target_sub, selected_oil_wels):
    '''
    Функция принимает на вход объём запасов в скважине (тыс. баррелей) из подвыборок бутстрепа:
    - спрогнозированный
    - фактический
    а также количество выбранных для разработки скважин
    '''
    # упорядочивание спрогнозированных значений по убыванию
    pred_sorted = predicted_sub.sort_values(ascending=False) 
    
    # отбор перспективных скважин
    selected_target = target_sub[pred_sorted.index][:selected_oil_wels]
    
    # прибыль для полученного объёма сырья:
    # суммирование целевых значений объёма сырья, соответствующее предсказаниям
    # домножение на доход с одной единицы продукта
    # вычитание общего бюджета
    # (величина может быть отрицательной, если доход меньше выделенного бюджета)
    return (selected_target.sum() * INCOME) - TOTAL_BUDGET

**п. 4.2**

Вероятность убытков и прибыль для каждого региона

In [12]:
def profit_bootstrap(predicted, target):
    '''
    Функция принимает на вход:
    - предсказания модели
    - метки валидационной выборки
    '''
    state = np.random.RandomState(12345)
    values = []
    
    for i in range(1000):
        target_subsample = target.sample(n=OIL_WELS, replace=True, random_state=state)
        predicted_subsample = predicted[target_subsample.index]
        
        res = profit(predicted_subsample, target_subsample, SELECTED_OIL_WELS)
        values.append(res)
        
    values = pd.Series(values)
    profit_avg = values.mean()
    
    # границы 95%-ого доверительного интервала
    lower = values.quantile(0.025) # ((100-95)/2)/100 = 0,025 
    upper = values.quantile(0.975) # 1-0,025 = 0,975
    
    loss_prob = values.lt(0).mean() # доля отрицательных значений (убытков)
    
    return profit_avg/10**6, lower/10**6, upper/10**6, loss_prob

In [13]:
# Вызов функции 'profit_bootstrap' для региона '1'
predicted_val_1 = (pd.Series(predicted_val_1)) # добавление индексов
target_val_1 = (target_val_1.reset_index(drop=True)) # назначение упорядоченных индексов
profit_avg_1, lower_1, upper_1, loss_prob_1 = profit_bootstrap(predicted_val_1, target_val_1)

# Вызов функции 'profit_bootstrap' для региона '2'
predicted_val_2 = (pd.Series(predicted_val_2)) # добавление индексов
target_val_2 = (target_val_2.reset_index(drop=True)) # назначение упорядоченных индексов
profit_avg_2, lower_2, upper_2, loss_prob_2 = profit_bootstrap(predicted_val_2, target_val_2)

# Вызов функции 'profit_bootstrap' для региона '3'
predicted_val_3 = (pd.Series(predicted_val_3)) # добавление индексов
target_val_3 = (target_val_3.reset_index(drop=True)) # назначение упорядоченных индексов
profit_avg_3, lower_3, upper_3, loss_prob_3 = profit_bootstrap(predicted_val_3, target_val_3)

for i in range(1, 4):
    print(f'Регион {i}')
    print(f'Средняя прибыль: {eval(f"profit_avg_{i}"):.1f} млн руб.')
    print(f'95% доверительный интервал: от {eval(f"lower_{i}"):.1f} до {eval(f"upper_{i}"):.1f} млн руб.')
    print(f'Вероятность убытков: {eval(f"loss_prob_{i}"):.1%}')
    print('='*30)

Регион 1
Средняя прибыль: 425.9 млн руб.
95% доверительный интервал: от -102.1 до 948.0 млн руб.
Вероятность убытков: 6.0%
Регион 2
Средняя прибыль: 515.2 млн руб.
95% доверительный интервал: от 68.9 до 931.5 млн руб.
Вероятность убытков: 1.0%
Регион 3
Средняя прибыль: 435.0 млн руб.
95% доверительный интервал: от -128.9 до 969.7 млн руб.
Вероятность убытков: 6.4%


### Вывод

По условия задания выбирается регион (или регионы) с вероятностью убытков менее 2.5%. Далее назначается регион с наибольшей средней прибылью.  

Указанным выше условиям соответствует только регион `2`. С показателями: 1.0% и 515.2 млн руб. соответственно.